In [1]:
import numpy as np
import xgboost as xgb
from wideboost.wrappers import wxgb
from wideboost.wrappers.wxgb import wxgbModel
from matplotlib import pyplot as plt

from sklearn import datasets

In [2]:
digits = datasets.load_digits()

idx_train = np.random.choice(np.arange(1797),round(1797*0.7),replace=False)
idx_test = np.setdiff1d(np.arange(1797),idx_train)


xtrain = digits['data']
ytrain = digits['target']

dtrain = xgb.DMatrix(xtrain[idx_train,:],label=ytrain[idx_train])
dtest = xgb.DMatrix(xtrain[idx_test,:],label=ytrain[idx_test])

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK, space_eval

best_val = 1.0

def objective(param):
    global best_val
    watchlist = [(dtrain,'train'),(dtest,'test')]
    ed1_results = dict()
    print(param)
    wbst = wxgb.train(param, dtrain, 100,watchlist,evals_result=ed1_results)
    
    output = min(ed1_results['test']['merror'])
    
    if output < best_val:
        print("NEW BEST VALUE!")
        best_val = output
    
    return {'loss': output, 'status': STATUS_OK }

spc = {
    'btype': hp.choice('btype',['I']),
    'extra_dims': hp.choice('extra_dims',[0]),
    'objective': hp.choice('objective',['multi:softmax']),
    'eval_metric':hp.choice('eval_metric',[['merror']]),
    'num_class':hp.choice('num_class',[10]),
    'eta': hp.loguniform('eta', -7, 0),
    'max_depth' : hp.choice('max_depth',range(1,11)),
    'subsample': hp.uniform('subsample', 0.5, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
    'colsample_bylevel': hp.uniform('colsample_bylevel', 0.5, 1),
    'min_child_weight': hp.loguniform('min_child_weight', -16, 5),
    'alpha': hp.choice('alpha', [0, hp.loguniform('alpha_positive', -16, 2)]),
    'lambda': hp.choice('lambda', [0, hp.loguniform('lambda_positive', -16, 2)]),
    'gamma': hp.choice('gamma', [0, hp.loguniform('gamma_positive', -16, 2)])
}

best = fmin(objective,
    space=spc,
    algo=tpe.suggest,
    max_evals=100)

{'alpha': 6.37600461167067e-05, 'btype': 'I', 'colsample_bylevel': 0.5739769418156129, 'colsample_bytree': 0.5504510431341878, 'eta': 0.33342803405748417, 'eval_metric': ('merror',), 'extra_dims': 0, 'gamma': 1.7353679050191434e-05, 'lambda': 0, 'max_depth': 5, 'min_child_weight': 8.707264347923768e-07, 'num_class': 10, 'objective': 'multi:softmax', 'subsample': 0.8545005799392069}
Overwriting param `num_class`                          
Overwriting param `objective` while setting `obj` in train.
Taking first argument of eval_metric. Multiple evals not supported using xgboost backend.
Moving param `eval_metric` to an feval.                
Setting param `disable_default_eval_metric` to 1.      
[10:00:20] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { btype } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  veri

[3]	train-merror:0.15819	test-merror:0.17625                           

[4]	train-merror:0.12719	test-merror:0.15399                           

[5]	train-merror:0.12242	test-merror:0.14100                           

[6]	train-merror:0.11129	test-merror:0.13729                           

[7]	train-merror:0.10016	test-merror:0.11874                           

[8]	train-merror:0.09459	test-merror:0.10761                           

[9]	train-merror:0.08664	test-merror:0.10390                           

[10]	train-merror:0.08426	test-merror:0.10019                          

[11]	train-merror:0.08029	test-merror:0.10019                          

[12]	train-merror:0.07870	test-merror:0.09276                          

[13]	train-merror:0.07393	test-merror:0.09462                          

[14]	train-merror:0.07313	test-merror:0.09462                          

[15]	train-merror:0.07075	test-merror:0.09648                          

[16]	train-merror:0.06995	test-merror:0.09648      

[0]	train-merror:0.89110	test-merror:0.91466                           

[1]	train-merror:0.89110	test-merror:0.91466                           

[2]	train-merror:0.89110	test-merror:0.91466                           

[3]	train-merror:0.89110	test-merror:0.91466                           

[4]	train-merror:0.89110	test-merror:0.91466                           

[5]	train-merror:0.89110	test-merror:0.91466                           

[6]	train-merror:0.89110	test-merror:0.91466                           

[7]	train-merror:0.89110	test-merror:0.91466                           

[8]	train-merror:0.89110	test-merror:0.91466                           

[9]	train-merror:0.89110	test-merror:0.91466                           

[10]	train-merror:0.89110	test-merror:0.91466                          

[11]	train-merror:0.89110	test-merror:0.91466                          

[12]	train-merror:0.89110	test-merror:0.91466                          

[13]	train-merror:0.89110	test-merror:0.91466      

[0]	train-merror:0.13514	test-merror:0.18367                           

[1]	train-merror:0.08823	test-merror:0.13729                           

[2]	train-merror:0.06995	test-merror:0.10019                           

[3]	train-merror:0.05723	test-merror:0.09648                           

[4]	train-merror:0.04929	test-merror:0.08163                           

[5]	train-merror:0.04690	test-merror:0.07421                           

[6]	train-merror:0.04531	test-merror:0.07978                           

[7]	train-merror:0.03975	test-merror:0.07421                           

[8]	train-merror:0.03577	test-merror:0.07236                           

[9]	train-merror:0.03577	test-merror:0.06308                           

[10]	train-merror:0.03100	test-merror:0.06679                          

[11]	train-merror:0.03339	test-merror:0.06865                          

[12]	train-merror:0.03021	test-merror:0.06494                          

[13]	train-merror:0.02782	test-merror:0.07050      

[0]	train-merror:0.00080	test-merror:0.13358                           

[1]	train-merror:0.00000	test-merror:0.09462                           

[2]	train-merror:0.00000	test-merror:0.06679                           

[3]	train-merror:0.00000	test-merror:0.05009                           

[4]	train-merror:0.00000	test-merror:0.05009                           

[5]	train-merror:0.00000	test-merror:0.04824                           

[6]	train-merror:0.00000	test-merror:0.04267                           

[7]	train-merror:0.00000	test-merror:0.04453                           

[8]	train-merror:0.00000	test-merror:0.03896                           

[9]	train-merror:0.00000	test-merror:0.03525                           

[10]	train-merror:0.00000	test-merror:0.02968                          

[11]	train-merror:0.00000	test-merror:0.02968                          

[12]	train-merror:0.00000	test-merror:0.03340                          

[13]	train-merror:0.00000	test-merror:0.02968      

...

[23]	train-merror:0.10016	test-merror:0.11317                           

[24]	train-merror:0.09698	test-merror:0.11503                           

[25]	train-merror:0.09857	test-merror:0.11317                           

[26]	train-merror:0.09698	test-merror:0.10946                           

[27]	train-merror:0.09459	test-merror:0.10761                           

[28]	train-merror:0.09062	test-merror:0.10575                           

[29]	train-merror:0.09062	test-merror:0.10204                           

[30]	train-merror:0.08903	test-merror:0.10204                           

[31]	train-merror:0.08983	test-merror:0.10019                           

[32]	train-merror:0.08823	test-merror:0.10019                           

[33]	train-merror:0.08823	test-merror:0.10575                           

[34]	train-merror:0.08585	test-merror:0.10390                           

[35]	train-merror:0.08426	test-merror:0.10390                           

[36]	train-merror:0.08506	test-merror:

[18]	train-merror:0.00000	test-merror:0.03525                           

[19]	train-merror:0.00000	test-merror:0.03711                           

[20]	train-merror:0.00000	test-merror:0.03525                           

[21]	train-merror:0.00000	test-merror:0.03525                           

[22]	train-merror:0.00000	test-merror:0.03340                           

[23]	train-merror:0.00000	test-merror:0.03711                           

[24]	train-merror:0.00000	test-merror:0.03340                           

[25]	train-merror:0.00000	test-merror:0.03340                           

[26]	train-merror:0.00000	test-merror:0.03340                           

[27]	train-merror:0.00000	test-merror:0.03340                           

[28]	train-merror:0.00000	test-merror:0.03340                           

[29]	train-merror:0.00000	test-merror:0.03340                           

[30]	train-merror:0.00000	test-merror:0.03340                           

[31]	train-merror:0.00000	test-merror:

[13]	train-merror:0.00000	test-merror:0.05195                           

[14]	train-merror:0.00000	test-merror:0.04824                           

[15]	train-merror:0.00000	test-merror:0.05195                           

[16]	train-merror:0.00000	test-merror:0.04453                           

[17]	train-merror:0.00000	test-merror:0.04638                           

[18]	train-merror:0.00000	test-merror:0.04267                           

[19]	train-merror:0.00000	test-merror:0.04082                           

[20]	train-merror:0.00000	test-merror:0.04267                           

[21]	train-merror:0.00000	test-merror:0.04082                           

[22]	train-merror:0.00000	test-merror:0.04267                           

[23]	train-merror:0.00000	test-merror:0.04453                           

[24]	train-merror:0.00000	test-merror:0.04453                           

[25]	train-merror:0.00000	test-merror:0.04082                           

[26]	train-merror:0.00000	test-merror:

[8]	train-merror:0.00000	test-merror:0.05566                            

[9]	train-merror:0.00000	test-merror:0.05009                            

[10]	train-merror:0.00000	test-merror:0.05009                           

[11]	train-merror:0.00000	test-merror:0.05009                           

[12]	train-merror:0.00000	test-merror:0.04824                           

[13]	train-merror:0.00000	test-merror:0.04638                           

[14]	train-merror:0.00000	test-merror:0.04824                           

[15]	train-merror:0.00000	test-merror:0.04267                           

[16]	train-merror:0.00000	test-merror:0.04453                           

[17]	train-merror:0.00000	test-merror:0.04453                           

[18]	train-merror:0.00000	test-merror:0.04267                           

[19]	train-merror:0.00000	test-merror:0.04267                           

[20]	train-merror:0.00000	test-merror:0.04082                           

[21]	train-merror:0.00000	test-merror:

[3]	train-merror:0.02146	test-merror:0.08349                            

[4]	train-merror:0.01590	test-merror:0.08534                            

[5]	train-merror:0.01510	test-merror:0.08534                            

[6]	train-merror:0.01192	test-merror:0.07421                            

[7]	train-merror:0.01033	test-merror:0.06865                            

[8]	train-merror:0.01033	test-merror:0.06494                            

[9]	train-merror:0.00874	test-merror:0.06308                            

[10]	train-merror:0.00954	test-merror:0.05937                           

[11]	train-merror:0.00874	test-merror:0.05751                           

[12]	train-merror:0.01033	test-merror:0.05566                           

[13]	train-merror:0.00715	test-merror:0.05380                           

[14]	train-merror:0.00636	test-merror:0.05566                           

[15]	train-merror:0.00556	test-merror:0.05937                           

[16]	train-merror:0.00556	test-merror:

In [4]:
print(best_val)
print(space_eval(spc, best))

0.014842
{'alpha': 0.0025213633137586796, 'btype': 'I', 'colsample_bylevel': 0.6887579427360749, 'colsample_bytree': 0.6656795837917295, 'eta': 0.11766894319241901, 'eval_metric': ('merror',), 'extra_dims': 0, 'gamma': 0.0005041931296086806, 'lambda': 0, 'max_depth': 5, 'min_child_weight': 0.00011944939314759394, 'num_class': 10, 'objective': 'multi:softmax', 'subsample': 0.6621504054409455}
